# Setup

## Import packages

In [2]:
from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, TileLayer, LayersControl
import json
import panel as pn
pn.extension("ipywidgets", sizing_mode="stretch_width")
import requests

## Declare variables

In [2]:
provider = 'NASA_MAAP'
cmr_search_url = "https://cmr.maap-project.org/search"
collections_search_url = f"{cmr_search_url}/collections.json?provider={provider}&page_size=100"
granules_search_url = f"{cmr_search_url}/collections.json"

## Make a selection of collections

In [3]:
response = requests.get(collections_search_url)
response

<Response [200]>

In [4]:
collections = json.loads(response.text)['feed']['entry']

In [1]:
collection_names = list(collection['short_name'] for collection in collections)

['blah']


In [6]:
# create a drop down of collections
select_collection = pn.widgets.Select(name='Select', options=[collection_names])

# Make a bbox selection

In [8]:
m = Map(
    basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
    center=(0, 0),
    zoom=4
)
draw_control = DrawControl()

draw_control.rectangle = {
    "shapeOptions": {
        "fillColor": "#fca45d",
        "color": "#fca45d",
        "fillOpacity": 0.2
    }
}
m.add_control(draw_control)

geojson = []

output_panel = pn.pane.Markdown("NOTHING DRAWN YET")

def handle_draw(self, action, geo_json):
    """Do something with the GeoJSON when it's drawn on the map"""
    print(geo_json)
    geojson.append(geo_json)
    output_panel.object = str(geo_json)

draw_control.on_draw(handle_draw)

map_panel = pn.Row(m, output_panel)

Row(sizing_mode='stretch_width')
    [0] IPyLeaflet(Map)
    [1] Markdown(str, sizing_mode='stretch_width')

In [9]:
bbox = geojson[0]['geometry']['coordinates'][0]
bbox_params = ','.join(map(str, [bbox[0][0], bbox[0][1], bbox[2][0], bbox[1][1]]))

In [10]:
bbox_params

'5.947037,-5.691149,17.548599,3.350293'

In [51]:
search_url_with_params = f"{cmr_search_url}/granules.json?short_name={select_collection.value}&bounding_box[]={bbox_params}"
search_url_with_params

'https://cmr.maap-project.org/search/granules.json?short_name=SRTMGL1_COD&bounding_box[]=5.947037,-5.691149,17.548599,3.350293'

# Search for granules and select a granule to visualize

In [52]:
# get all granules from search
granules_response = requests.get(search_url_with_params)

<Response [200]>

In [53]:
granules_objects = json.loads(granules_response.text)['feed']['entry']
granule_urls = list(granule['links'][0]['href'] for granule in granules_objects)

In [54]:
# create a drop down of collections
select_granules = pn.widgets.Select(name='Select', options=[granule_urls])

dashboard = pn.Column(
    map_panel,
    pn.Row(select_granules),
    pn.Row(select_collection)
)
select_granules.serveable()

Row(sizing_mode='stretch_width')
    [0] Select(options=['s3://nasa-maap-data-stor...], sizing_mode='stretch_width', value='s3://nasa-maap-data-store...)

In [62]:
def update_map():
    granule_s3_url = select_granules.value
    #tititler_url = "https://l40l1vwmvi.execute-api.us-west-2.amazonaws.com"
    rescale = "0,400"
    colormap_name = "gist_earth_r"
    color_formula = "gamma r 2"
    tititler_url = "https://titiler.maap-project.org"
    #tiles_url = tititler_url + "/cog/tiles/{z}/{x}/{y}.png?url=" + f"{granule_s3_url}&rescale={rescale}&colormap_name={colormap_name}&color_formula={color_formula}"
    tiles_url = tititler_url + "/cog/tiles/{z}/{x}/{y}.png?url=" + f"{granule_s3_url}&rescale=350,850&colormap_name={colormap_name}"
    print(tiles_url)
    tile_layer=TileLayer(url=tiles_url)

    m = Map(
        basemap=basemap_to_tiles(basemaps.OpenStreetMap.Mapnik),
        center=(0,0),
        zoom=5)

    m.add_layer(tile_layer)
    m.add_control(LayersControl())
    return m

In [63]:
# update_map()

https://titiler.maap-project.org/cog/tiles/{z}/{x}/{y}.png?url=s3://nasa-maap-data-store/file-staging/nasa-map/SRTMGL1_COD___001/N02E014.SRTMGL1.tif&rescale=350,850&colormap_name=gist_earth_r


Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…